In [1]:
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import multiprocessing

from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec
import time
import datetime

In [14]:
print("Starting")
print("Wiki loaded")
print(datetime.datetime.now())
print(datetime.datetime.now() - start)
start = datetime.datetime.now()

Starting
Wiki loaded
2020-01-26 15:52:30.225856
0:00:28.291065


In [ ]:
wiki = WikiCorpus("./data/enwiki-latest-pages-articles1.xml-p10p30302.bz2")

In [4]:
class TaggedWikiDocument(object):
    def __init__(self, wiki):
        self.wiki = wiki
        self.wiki.metadata = True
    def __iter__(self):
        for content, (page_id, title) in self.wiki.get_texts():
            yield TaggedDocument(content, [title])
            
class EpochLoggerDBOW(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [4]:
documents = TaggedWikiDocument(wiki)

In [8]:
pre = Doc2Vec(min_count=0)

In [9]:
# pre = Doc2Vec(min_count=0)
# pre.build_vocab(documents)

In [10]:
# for num in range(0, 20):
#     print('min_count: {}, size of vocab: '.format(num), pre.scale_vocab(min_count=num, dry_run=True)['memory']['vocab']/700)

In [11]:
cores = multiprocessing.cpu_count()

models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, size=200, window=8, min_count=19, iter=10, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=200, window=8, min_count=19, iter =10, workers=cores),
]

/home/ubuntu/anaconda3/envs/myenv/lib/python3.6/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/home/ubuntu/anaconda3/envs/myenv/lib/python3.6/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [12]:
models[0].build_vocab(documents)
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

Doc2Vec(dbow+w,d200,n5,w8,mc19,s0.001,t16)
Doc2Vec(dm/m,d200,n5,w8,mc19,s0.001,t16)


In [ ]:
for model in models:
    model.train(documents, total_examples=model.corpus_count, epochs=model.iter)

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


In [5]:
from gensim.test.utils import get_tmpfile
temp_path_dbow = get_tmpfile('DBOW_epoch7.model')
# models[0].save(temp_path_dbow)
# temp_path_db = get_tmpfile('wiki_d2v_db')
# models[1].save(temp_path_db)

model_dbow = Doc2Vec.load(temp_path_dbow)
# model_db = Doc2Vec.load(temp_path_db)


In [6]:
# similarity
# for model in models:
#     print(str(model))
#     pprint(model.docvecs.most_similar(positive=["Machine learning"], topn=20))

pprint(model_dbow.docvecs.most_similar(positive=["Machine learning"], topn=20))
# pprint(model_db.docvecs.most_similar(positive=["Machine learning"], topn=20))

[('Supervised learning', 0.6636233925819397),
 ('Pattern recognition', 0.6623779535293579),
 ('Outline of machine learning', 0.6366842985153198),
 ('Deep learning', 0.6187774538993835),
 ('Artificial neural network', 0.612984836101532),
 ('Unsupervised learning', 0.6096498966217041),
 ('Feature selection', 0.6061097383499146),
 ('Boosting (machine learning)', 0.6051622033119202),
 ('Predictive analytics', 0.6042125821113586),
 ('Data mining', 0.602078378200531),
 ('Artificial intelligence', 0.592949390411377),
 ('Bayesian network', 0.5902780294418335),
 ('Linear classifier', 0.585090160369873),
 ('Outline of computer science', 0.5741325616836548),
 ('Symbolic artificial intelligence', 0.5736307501792908),
 ('Glossary of artificial intelligence', 0.5702601671218872),
 ('Deeplearning4j', 0.5696315765380859),
 ('Early stopping', 0.5691721439361572),
 ('Latent semantic analysis', 0.568435788154602),
 ('Neural Designer', 0.5665047764778137)]


NameError: name 'model_db' is not defined

In [68]:
text = 'Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to perform a specific task without using explicit instructions, relying on patterns and inference instead. It is seen as a subset of artificial intelligence. Machine learning algorithms build a mathematical model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task.[1][2]:2 Machine learning algorithms are used in a wide variety of applications, such as email filtering and computer vision, where it is difficult or infeasible to develop a conventional algorithm for effectively performing the task.'
text1 = 'Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to perform a specific task without using explicit instructions, relying on patterns and inference instead'
text2 = 'The name machine learning was coined in 1959 by Arthur Samuel.[5] Tom M. Mitchell provided a widely quoted, more formal definition of the algorithms studied in the machine learning field: "A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P if its performance at tasks in T, as measured by P, improves with experience E."[6] This definition of the tasks in which machine learning is concerned offers a fundamentally operational definition rather than defining the field in cognitive terms. This follows Alan Turing\'s proposal in his paper "Computing Machinery and Intelligence", in which the question "Can machines think?" is replaced with the question "Can machines do what we (as thinking entities) can do?".[7] In Turing\'s proposal the various characteristics that could be possessed by a thinking machine and the various implications in constructing one are exposed.'
text3 = 'Implicit computational complexity (ICC) is a subfield of computational complexity theory that characterizes algorithms by constraints on the way in which they are constructed, without reference to a specific underlying machine model or to explicit bounds on computational resources unlike conventional complexity theory. ICC was developed in the 1990s and employs the techniques of proof theory, substructural logic, model theory and recursion theory to prove bounds on the expressive power of high-level formal languages. ICC is also concerned with the practical realization of functional programming languages, language tools and type theory that can control the resource usage of programs in a formally verifiable sense.'
text4 = 'Quantum artificial intelligence (QAI) is an interdisciplinary field that focuses on building quantum algorithms for improving computational tasks within artificial intelligence, including sub-fields like machine learning.Quantum mechanics phenomena, superposition and entanglement, are allowing quantum computing to perform computations which are much more efficient than classical AI algorithms used in computer vision, natural language processing and robotics.[1] The entire concept of quantum-enhanced AI algorithms is still in conceptual research domain. Building on recent theoretical proposals, initial practical studies suggest that these concepts have the possibility to be implemented in the laboratory, under strictly controlled conditions.[2]'
text5='Applying classical methods of machine learning to the study of quantum systems (sometimes called quantum machine learning) is the focus of an emergent area of physics research. A basic example of this is quantum state tomography, where a quantum state is learned from measurement. Other examples include learning Hamiltonians[1], learning quantum phase transitions[2][3], and automatically generating new quantum experiments[4][5][6][7]. Classical machine learning is effective at processing large amounts of experimental or calculated data in order to characterize an unknown quantum system, making its application useful in contexts including quantum information theory, quantum technologies development, and computational materials design.'
text6='MatrixNet is a proprietary machine learning algorithm developed by Yandex and used widely throughout the company products. The algorithm is based on gradient boosting and was introduced since 2009.[1][2][3]'
text7='Yandex N.V. (/ˈjʌndɛks/; Russian: Яндекс, IPA: [ˈjandəks]) is a Russian multinational corporation specializing in Internet-related products and services, including transportation, search and information services, eCommerce, navigation, mobile applications, and online advertising. Yandex provides over 70 services in total.[5][6] Incorporated in the Netherlands, Yandex primarily serves audiences in Russia and the Commonwealth of Independent States. The company founders and most of the team members are located in Russia. The company has 18 commercial offices worldwide.[7][8] It is the largest technology company in Russia[9] and the largest search engine on the internet in Russian, with a market share of over 52%.[10] The Yandex.ru home page is the 4th most popular website in Russia.[11] It also has the largest market share of any search engine in the Commonwealth of Independent States and is the 5th largest search engine worldwide after Google, Baidu, Bing, and Yahoo!.'
text8='In the nervous system, a synapse[2] is a structure that permits a neuron (or nerve cell) to pass an electrical or chemical signal to another neuron or to the target effector cell.Santiago Ramón y Cajal proposed that neurons are not continuous throughout the body, yet still communicate with each other, an idea known as the neuron doctrine.[3] The word "synapse" – from the Greek synapsis (συνάψις), meaning "conjunction", in turn from συνάπτεὶν (συν ("together") and ἅπτειν ("to fasten")) – was introduced in 1897 by the English neurophysiologist Charles Sherrington in Michael Foster\'s Textbook of Physiology.[2] Sherrington struggled to find a good term that emphasized a union between two separate elements, and the actual term "synapse" was suggested by the English classical scholar Arthur Woollgar Verrall, a friend of Foster.[4][5] Some authors generalize the concept of the synapse to include the communication from a neuron to any other cell type,[6] such as to a motor cell, although such non-neuronal contacts may be referred to as junctions (a historically older term).A landmark study by Sanford Palay demonstrated the existence of synapses.[7]Synapses are essential to neuronal function: neurons are cells that are specialized to pass signals to individual target cells, and synapses are the means by which they do so. At a synapse, the plasma membrane of the signal-passing neuron (the presynaptic neuron) comes into close apposition with the membrane of the target (postsynaptic) cell. Both the presynaptic and postsynaptic sites contain extensive arrays of molecular machinery that link the two membranes together and carry out the signaling process. In many synapses, the presynaptic part is located on an axon and the postsynaptic part is located on a dendrite or soma. Astrocytes also exchange information with the synaptic neurons, responding to synaptic activity and, in turn, regulating neurotransmission.[8] Synapses (at least chemical synapses) are stabilized in position by synaptic adhesion molecules (SAMs) projecting from both the pre- and post-synaptic neuron and sticking together where they overlap; SAMs may also assist in the generation and functioning of synapses.[9]'
text9 = 'Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python\'s design philosophy emphasizes code readability with its notable use of significant whitespace. Its language constructs and object-oriented approach aim to help programmers write clear, logical code for small and large-scale projects.[28]Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is often described as a "batteries included" language due to its comprehensive standard library.[29]'
text10 = 'Python was conceived in the late 1980s[34] by Guido van Rossum at Centrum Wiskunde & Informatica (CWI) in the Netherlands as a successor to the ABC language (itself inspired by SETL),[35] capable of exception handling and interfacing with the Amoeba operating system.[8] Its implementation began in December 1989.[36] Van Rossum shouldered sole responsibility for the project, as the lead developer, until 12 July 2018, when he announced his "permanent vacation" from his responsibilities as Python\'s Benevolent Dictator For Life, a title the Python community bestowed upon him to reflect his long-term commitment as the project\'s chief decision-maker.[37] He now shares his leadership as a member of a five-person steering council.[38][39][40] In January 2019, active Python core developers elected Brett Cannon, Nick Coghlan, Barry Warsaw, Carol Willing and Van Rossum to a five-member "Steering Council" to lead the project.[41]Python 2.0 was released on 16 October 2000 with many major new features, including a cycle-detecting garbage collector and support for Unicode.[42]Python 3.0 was released on 3 December 2008. It was a major revision of the language that is not completely backward-compatible.[43] Many of its major features were backported to Python 2.6.x[44] and 2.7.x version series. Releases of Python 3 include the 2to3 utility, which automates (at least partially) the translation of Python 2 code to Python 3.[45]'
text11 = 'You don\'t mention the size of your dataset - in rows, total words, unique words, or unique classes. Doc2Vec works best with lots of data. Most published work trains on tens-of-thousands to millions of documents, of dozens to thousands of words each. (Your data appears to only have 3-5 words per document.)Also, published work tends to train on data where every document has a unique-ID. It can sometimes make sense to use known-labels as tags instead of, or in addition to, unique-IDs. But it isn\'t necessarily a better approach. By using known-labels as the only tags, you\'re effectively only training one doc-vector per label. (It\'s essentially similar to concatenating all rows with the same tag into one document.)'
text12 = 'Multisignature (multi-signature) is a digital signature scheme which allows a group of users to sign a single document. Usually, a multisignature algorithm produces a joint signature that is more compact than a collection of distinct signatures from all users.[1]Multisignature can be considered as generalization of both group and ring signatures.Multisignature adds additional security for cryptocurrency transactions.'
text13 = 'Probability is a numerical description of how likely an event is to occur or how likely it is that a proposition is true. Probability is a number between 0 and 1, where, roughly speaking, 0 indicates impossibility and 1 indicates certainty.[note 1][1][2] The higher the probability of an event, the more likely it is that the event will occur. A simple example is the tossing of a fair (unbiased) coin. Since the coin is fair, the two outcomes ("heads" and "tails") are both equally probable; the probability of "heads" equals the probability of "tails"; and since no other outcomes are possible, the probability of either "heads" or "tails" is 1/2 (which could also be written as 0.5 or 50%).'
pprint(model_dbow.docvecs.most_similar([model_dbow.infer_vector(text13.split())], topn=20))
pprint(model_dbow.similar_by_vector(model_dbow.infer_vector(text13.split()), topn=20))

[('Effective sample size', 0.6514292359352112),
 ('Less-is-more effect', 0.6494983434677124),
 ('Maximum common subgraph', 0.6433039903640747),
 ('S-procedure', 0.6417062282562256),
 ("Cohen's h", 0.6371748447418213),
 ('Intensity (measure theory)', 0.6349696516990662),
 ('Predictive mean matching', 0.6315295696258545),
 ('Accuracy paradox', 0.6308474540710449),
 ('Numerical value equation', 0.628330409526825),
 ('Dichotomous thinking', 0.6268120408058167),
 ('Sub-probability measure', 0.6241165399551392),
 ('Elbow method (clustering)', 0.6229354739189148),
 ("Bendixson's inequality", 0.6215020418167114),
 ('C+-probability', 0.6206342577934265),
 ('Heptadiagonal matrix', 0.6195244789123535),
 ('Intensity measure', 0.6192264556884766),
 ('Binomial process', 0.6188932657241821),
 ('Indicator vector', 0.6173559427261353),
 ('Simple point process', 0.6162580251693726),
 ('Event structure', 0.6128488779067993)]
[('estimand', 0.5939438939094543),
 ('discrepancy', 0.5844589471817017),
 ('prob

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  app.launch_new_instance()


In [61]:
pprint(model_dbow.wv.most_similar(positive=["country"], topn=20))

[('time', 0.5998595952987671),
 ('run', 0.577658474445343),
 ('rank', 0.5759809017181396),
 ('place', 0.5707610845565796),
 ('event', 0.5684378147125244),
 ('name', 0.5597543716430664),
 ('world', 0.5573016405105591),
 ('final', 0.5564603805541992),
 ('cross', 0.5534589290618896),
 ('girls', 0.5459471940994263),
 ('individual', 0.5344840288162231),
 ('did', 0.5329700708389282),
 ('women', 0.5327101945877075),
 ('number', 0.5314116477966309),
 ('events', 0.5307267308235168),
 ('same', 0.5268730521202087),
 ('started', 0.5253022313117981),
 ('third', 0.5245479345321655),
 ('last', 0.5243156552314758),
 ('references', 0.523186445236206)]


In [39]:
pprint(model_dbow.vocabulary.make_cum_table(model_dbow.wv))

None
